In [11]:
# to save data from Vedomosti RSS to xml file to the same directory as the script placed
import csv
import requests
import xml.etree.ElementTree as ET
import datetime
import time
import os


In [12]:
#date and time for file name creation
datetime = datetime.datetime.now().strftime('%d%m%Y%H%M%S') 
datetime

'30122022131249'

In [13]:
#saving xml file with data from Lenta RSS
response = requests.get("https://lenta.ru/rss/")
with open(f'lenta{datetime}.xml', "w", encoding="utf-8") as f:
    f.write(response.text)

In [14]:
#to add data from xml file to filesdata table 
import glob
import psycopg2

#the latest file name 
list_of_files = glob.glob('*.xml') 
latest_file = max(list_of_files, key=os.path.getmtime)
print(latest_file)

lenta30122022131249.xml


In [15]:
#data from xml file
with open(latest_file, encoding='utf8') as f:
    contents = f.read()
    print(contents)

<?xml version="1.0" encoding="UTF-8"?>
<rss version="2.0" xmlns:atom="http://www.w3.org/2005/Atom">
  <channel>
    <language>ru</language>
    <title>Lenta.ru : Новости</title>
    <description>Новости, статьи, фотографии, видео. Семь дней в неделю, 24 часа в сутки.</description>
    <link>https://lenta.ru</link>
    <image>
      <url>https://lenta.ru/images/small_logo.png</url>
      <title>Lenta.ru</title>
      <link>https://lenta.ru</link>
      <width>134</width>
      <height>22</height>
    </image>
    <atom:link rel="self" type="application/rss+xml" href="http://lenta.ru/rss"/>
<item>
  <guid>https://lenta.ru/news/2022/12/30/china_russia/</guid>
  <author>Фариза Бацазова</author>
  <title>В России заявили о готовности возобновить пассажирское сообщение с Китаем</title>
  <link>https://lenta.ru/news/2022/12/30/china_russia/</link>
  <description>
    <![CDATA[Россия готова оперативно возобновить пассажирское сообщение с Китаем при советующем обращении Пекина. Об этом заявила 

In [16]:
#insert data from xml file to filesdata table
try:
    connection = psycopg2.connect(user="postgres",
                                  password="postgres",
                                  host="localhost",
                                  port="5432",
                                  database="news1")
    cursor = connection.cursor()

    postgres_insert = """ INSERT INTO filesdata (filename, xmldata) VALUES (%s,%s)"""
    data_to_insert = (f'{latest_file}', f'{contents}')
    cursor.execute(postgres_insert, data_to_insert)

    connection.commit()
    count = cursor.rowcount
    print(count, "record inserted successfully")

except (Exception, psycopg2.Error) as error:
    print("failed to insert", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

1 record inserted successfully
PostgreSQL connection is closed


In [17]:
#insert categories

try:
    connection = psycopg2.connect(user="postgres",
                                  password="postgres",
                                  host="localhost",
                                  port="5432",
                                  database="news1")
    cursor = connection.cursor()

    postgres_insert = """ WITH n AS (SELECT distinct text(UNNEST(xpath('//item/category/text()', xmldata))) AS category
FROM filesdata WHERE filename LIKE 'lenta%')
INSERT INTO categories (categoryname)
SELECT category FROM n
LEFT JOIN categories ON n.category=categories.categoryname
where category_id is NULL;"""

    cursor.execute(postgres_insert)
    
    connection.commit()
    count = cursor.rowcount
    print(count, "record inserted successfully")

except (Exception, psycopg2.Error) as error:
    print("failed to insert", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

0 record inserted successfully
PostgreSQL connection is closed


In [18]:
#insert news

try:
    connection = psycopg2.connect(user="postgres",
                                  password="postgres",
                                  host="localhost",
                                  port="5432",
                                  database="news1")
    cursor = connection.cursor()

    postgres_insert = f""" WITH q AS (WITH w AS (WITH n AS (SELECT text(UNNEST(xpath('//item/guid/text()', xmldata))) AS guid,
	   text(UNNEST(xpath('//item/author/text()', xmldata))) AS author,
	   text(UNNEST(xpath('//item/title/text()', xmldata))) AS title,
	   text(UNNEST(xpath('//item/link/text()', xmldata))) AS link,
	   RTRIM(LTRIM(text(UNNEST(xpath('//item/description[1]', xmldata))), '"<description>
    <![CDATA['), ']]>
  </description>') AS description,
	   cast(text(UNNEST(xpath('//item/pubDate/text()', xmldata))) AS timestamp) AS pubDate,
	   text(UNNEST(xpath('//item/enclosure/@url', xmldata))) AS enclosure,
	   text(UNNEST(xpath('//item/category/text()', xmldata))) AS category
FROM filesdata
WHERE filename='{latest_file}')
SELECT n.*, c.category_id FROM n
JOIN categories c ON n.category = c.categoryname)
SELECT w.* FROM w 
LEFT JOIN news nn ON w.guid = nn.guid
WHERE nn.guid is NULL)
	INSERT INTO news ( title, link, guid, pubdate, description, author, enclosure, pdalink, source_id, file_id)
SELECT  q.title, q.link, q.guid, q.pubdate, q.description, q.author, q.enclosure, NULL, '2',
(SELECT file_id FROM filesdata WHERE filename ='{latest_file}') FROM q"""

 
    cursor.execute(postgres_insert)
    
    connection.commit()
    count = cursor.rowcount
    print(count, "record inserted successfully")

except (Exception, psycopg2.Error) as error:
    print("failed to insert", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

40 record inserted successfully
PostgreSQL connection is closed


In [19]:
#insert newscategories

try:
    connection = psycopg2.connect(user="postgres",
                                  password="postgres",
                                  host="localhost",
                                  port="5432",
                                  database="news1")
    cursor = connection.cursor()

    postgres_insert = f""" INSERT INTO newscategories (news_id, category_id)
WITH a AS (WITH o AS (WITH l AS (SELECT text(UNNEST(xpath('//item/guid/text()', xmldata))) AS guid,
	   text(UNNEST(xpath('//item/category/text()', xmldata))) AS category
FROM filesdata
WHERE filename='{latest_file}')
SELECT n.news_id, n.guid, l.category
FROM news n
JOIN l ON n.guid = l.guid)
SELECT o.*, c.category_id FROM o
JOIN categories c ON o.category = c.categoryname)
SELECT a.news_id, a.category_id FROM a 
LEFT JOIN newscategories nc ON a.news_id = nc.news_id
where nc.news_id is NULL"""
   
    cursor.execute(postgres_insert)
    
    connection.commit()
    count = cursor.rowcount
    print(count, "record inserted successfully")

except (Exception, psycopg2.Error) as error:
    print("failed to insert", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

40 record inserted successfully
PostgreSQL connection is closed
